<a href="https://colab.research.google.com/github/Edgar-La/Intro_to_Natural_Language_Processing/blob/main/Project1/TM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bird, Steven, Edward Loper and Ewan Klein (2009), Natural Language Processing with Python. O’Reilly Media Inc.

# Descargar el archivo a analizar

In [21]:
!gdown --id 1A0ijJSvPRO3DPX_O50dsmq4EsEWaDGxc

Downloading...
From: https://drive.google.com/uc?id=1A0ijJSvPRO3DPX_O50dsmq4EsEWaDGxc
To: /content/Datasets - Comments.xlsx
100% 81.5k/81.5k [00:00<00:00, 73.6MB/s]


# Importar paquetes

## Pre-procesamiento

In [22]:
import pandas as pd
import numpy as np
!pip install langdetect
!pip install iso-639
from langdetect import detect
from iso639 import languages
!pip install langcodes
!pip install language_data
import langcodes
from textblob import TextBlob
!pip install twython
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import altair as alt
import numpy as np

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Se guarda la hoja de excel como dataframe

In [23]:
df = pd.read_excel('Datasets - Comments.xlsx', sheet_name = 'TripAdvisor')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,Rango de fecha de comentarios,NaN,NaN,2015 - 2021,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,No,Name (click to view profile),Crítico de nivel,City,Profile ID,Date,Likes,Comment,View Source
3,NaN,1,Cipriano N,2,Orizaba,-,23 de marzo de 2016,NaN,El lugar es genial para comer hasta que te dej...,NaN
4,NaN,2,Oaxacalife,3,Orizaba,NaN,5 de junio de 2016,NaN,Limpio y bien diseñado. Bebidas y maravillosa ...,NaN


### Hay columnas que parecen sospechosas, lucen vacias entonces verificamos que en efecto lo sean

In [24]:
print(df['Unnamed: 0'].unique())
print(df['Unnamed: 5'].unique())
print(df['Unnamed: 7'].unique())
print(df['Unnamed: 9'].unique())
#print(df['Unnamed: 10'].unique())
#print(df['Unnamed: 11'].unique())
#print(df['Unnamed: 12'].unique())

[nan]
['Rango de fecha de comentarios' nan 'Profile ID' '-']
[nan 'Likes']
[nan 'View Source']


### Eliminamos esas columnas>

In [25]:
df.drop(columns = ['Unnamed: 0',
                   'Unnamed: 1',
                   'Unnamed: 5',
                   'Unnamed: 7',
                   'Unnamed: 9',
                   #'Unnamed: 10',
                   #'Unnamed: 11',
                   #'Unnamed: 12'
                   ],
                   axis = 1, inplace = True)
df.head()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 6,Unnamed: 8
0,NaN,NaN,NaN,NaN,2015 - 2021
1,NaN,NaN,NaN,NaN,NaN
2,Name (click to view profile),Crítico de nivel,City,Date,Comment
3,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...
4,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...


### Hay una fila (la segunda) que contiene el nombre las columnas, la obtenemos porque vamos a renombrar las columnas

In [26]:
columnas = df[df.columns].values[2]
columnas

array(['Name (click to view profile)', 'Crítico de nivel', 'City', 'Date',
       'Comment'], dtype=object)

### Eliminamos las filas que no contienen información importante y que eran parte del excel

In [27]:
df.drop([0,1,2], axis=0, inplace=True)
df.head()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 6,Unnamed: 8
3,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...
4,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...
5,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped..."
6,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...
7,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue..."


### Eliminamos el nombre de las columnas (para poder reasignar el nombre correcto)

In [28]:
df.columns = [''] * len(df.columns)
df.head()

,,,,,
3,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...
4,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...
5,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped..."
6,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...
7,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue..."


### Reasignamos los nombres correctos a las columnas

In [29]:
df.columns = columnas
df.rename(columns = {'Name (click to view profile)':'Name'}, inplace = True)
df.head()

,Name,Crítico de nivel,City,Date,Comment
3,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...
4,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...
5,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped..."
6,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...
7,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue..."


### Por ultimo, reseteamos el indice del dataframe

In [30]:
df.reset_index(inplace = True)
df.drop(columns = ['index'], axis=1, inplace=True)
df.head()

,Name,Crítico de nivel,City,Date,Comment
0,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...
1,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...
2,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped..."
3,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...
4,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue..."


## Agregamos una columna que nos regresa solamente el año del comentario para hacer un grafico interesante mas adelante

In [31]:
Year = []
for i in range(len(df)):
  Year.append(df['Date'][i][-4:])
df['Year'] = Year
df.head()

,Name,Crítico de nivel,City,Date,Comment,Year
0,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...,2016
1,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...,2016
2,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped...",2017
3,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...,2017
4,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue...",2017


# NLP

## Creamos un nueva columna en el dataframe que indica el idioma del comentario

In [32]:
Language = []
for c in df['Comment']:
    Language.append(languages.get(alpha2 = detect(c)).name.lower())
df['Language'] = Language
df

,Name,Crítico de nivel,City,Date,Comment,Year,Language
0,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...,2016,spanish
1,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...,2016,spanish
2,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped...",2017,spanish
3,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...,2017,spanish
4,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue...",2017,spanish
...,...,...,...,...,...,...,...
150,NormanT708,4,Orizaba,29 de diciembre de 2019,I had previously reviewed other Madison grill ...,2019,english
151,cortegas72,4,Orizaba,16 de diciembre de 2020,Great restaurant to have a pleasant time with ...,2020,english
152,Pablo R,3,Orizaba,24 de enero de 2016,Das Restaurant ist sehr schön an einem Platz g...,2016,german
153,Xavigrand,4,Orizaba,18 de julio de 2018,"Si vous êtes un fana de viandes, ce restaurant...",2018,french


## Creamos un nueva columna en el dataframe que indica la abreviatura del comentario en iso-639

In [33]:
lang_iso639 = []
for lang in Language:
  lang_iso639.append(langcodes.find(lang).language)
df['Lang_iso639'] = lang_iso639
df

,Name,Crítico de nivel,City,Date,Comment,Year,Language,Lang_iso639
0,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...,2016,spanish,es
1,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...,2016,spanish,es
2,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped...",2017,spanish,es
3,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...,2017,spanish,es
4,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue...",2017,spanish,es
...,...,...,...,...,...,...,...,...
150,NormanT708,4,Orizaba,29 de diciembre de 2019,I had previously reviewed other Madison grill ...,2019,english,en
151,cortegas72,4,Orizaba,16 de diciembre de 2020,Great restaurant to have a pleasant time with ...,2020,english,en
152,Pablo R,3,Orizaba,24 de enero de 2016,Das Restaurant ist sehr schön an einem Platz g...,2016,german,de
153,Xavigrand,4,Orizaba,18 de julio de 2018,"Si vous êtes un fana de viandes, ce restaurant...",2018,french,fr


### Mostramos todos los idiomas en los que estan escritos los comentarios para darnos una idea

In [34]:
unique_values_language = df['Language'].unique()
unique_values_language

array(['spanish', 'swedish', 'english', 'german', 'french', 'italian'],
      dtype=object)

## Traducimos a ingles todos los comentarios para poder usar NLTK y analisis de sentimientos mas adelante

In [35]:
translated_comment = []
for i in range(len(df['Comment'])):
  if df['Lang_iso639'][i] == 'en':
    translated_comment.append(df['Comment'][i])
  else:
    translated_comment.append(str(TextBlob(df['Comment'][i]).translate(from_lang = df['Lang_iso639'][i], to='en')))
df['Translated_comment'] = translated_comment
df

,Name,Crítico de nivel,City,Date,Comment,Year,Language,Lang_iso639,Translated_comment
0,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...,2016,spanish,es,The place is great to eat until it will leave ...
1,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...,2016,spanish,es,Clean and well designed. Drinks and wonderful ...
2,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped...",2017,spanish,es,"Very good meat restaurant, do not hesitate to ..."
3,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...,2017,spanish,es,This is a place where you can eat the classic ...
4,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue...",2017,spanish,es,"The food was delicious, I ate a hamburger and ..."
...,...,...,...,...,...,...,...,...,...
150,NormanT708,4,Orizaba,29 de diciembre de 2019,I had previously reviewed other Madison grill ...,2019,english,en,I had previously reviewed other Madison grill ...
151,cortegas72,4,Orizaba,16 de diciembre de 2020,Great restaurant to have a pleasant time with ...,2020,english,en,Great restaurant to have a pleasant time with ...
152,Pablo R,3,Orizaba,24 de enero de 2016,Das Restaurant ist sehr schön an einem Platz g...,2016,german,de,The restaurant is very nice in a place. We esp...
153,Xavigrand,4,Orizaba,18 de julio de 2018,"Si vous êtes un fana de viandes, ce restaurant...",2018,french,fr,"If you are a fan of meats, this restaurant is ..."


## A partir de los comentarios traducidos, generamos los _scores_ y asignamos la clase correspondiente

In [36]:
Polapolarity_scores = []
Label = []
sid = SentimentIntensityAnalyzer()
for i in range(len(df['Translated_comment'])):
    ss = sid.polarity_scores(df['Translated_comment'][i])
    Polapolarity_scores.append(ss)
    if ss['compound'] > 0.05:
      Label.append('positive')
    elif ss['compound'] < -0.05:
      Label.append('negative')
    elif (ss['compound'] >= -0.05) & (ss['compound'] <= 0.05):
      Label.append('neutral')
df['Polapolarity_scores'] = Polapolarity_scores
df['Label'] = Label
df

,Name,Crítico de nivel,City,Date,Comment,Year,Language,Lang_iso639,Translated_comment,Polapolarity_scores,Label
0,Cipriano N,2,Orizaba,23 de marzo de 2016,El lugar es genial para comer hasta que te dej...,2016,spanish,es,The place is great to eat until it will leave ...,"{'neg': 0.048, 'neu': 0.841, 'pos': 0.11, 'com...",positive
1,Oaxacalife,3,Orizaba,5 de junio de 2016,Limpio y bien diseñado. Bebidas y maravillosa ...,2016,spanish,es,Clean and well designed. Drinks and wonderful ...,"{'neg': 0.0, 'neu': 0.576, 'pos': 0.424, 'comp...",positive
2,AugustoLara,2,Orizaba,3 de febrero 2017,"Muy buen restaurante de carnes, no dude en ped...",2017,spanish,es,"Very good meat restaurant, do not hesitate to ...","{'neg': 0.02, 'neu': 0.726, 'pos': 0.253, 'com...",positive
3,GABINO M,4,Orizaba,24 de mayo de 2017,Este es un lugar donde se puede comer los taco...,2017,spanish,es,This is a place where you can eat the classic ...,"{'neg': 0.0, 'neu': 0.76, 'pos': 0.24, 'compou...",positive
4,MALH2001,-,Orizaba,15 de junio de 2017,"La comida era deliciosa, me comí una hamburgue...",2017,spanish,es,"The food was delicious, I ate a hamburger and ...","{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'comp...",positive
...,...,...,...,...,...,...,...,...,...,...,...
150,NormanT708,4,Orizaba,29 de diciembre de 2019,I had previously reviewed other Madison grill ...,2019,english,en,I had previously reviewed other Madison grill ...,"{'neg': 0.033, 'neu': 0.574, 'pos': 0.393, 'co...",positive
151,cortegas72,4,Orizaba,16 de diciembre de 2020,Great restaurant to have a pleasant time with ...,2020,english,en,Great restaurant to have a pleasant time with ...,"{'neg': 0.0, 'neu': 0.639, 'pos': 0.361, 'comp...",positive
152,Pablo R,3,Orizaba,24 de enero de 2016,Das Restaurant ist sehr schön an einem Platz g...,2016,german,de,The restaurant is very nice in a place. We esp...,"{'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'comp...",positive
153,Xavigrand,4,Orizaba,18 de julio de 2018,"Si vous êtes un fana de viandes, ce restaurant...",2018,french,fr,"If you are a fan of meats, this restaurant is ...","{'neg': 0.0, 'neu': 0.832, 'pos': 0.168, 'comp...",positive


# Generamos un grafico con la informacion obtenida

In [37]:
df2 = df['Label'].value_counts().reset_index()
df2.columns = ['Labels', 'Counts']
df2

,Labels,Counts
0,positive,142
1,negative,10
2,neutral,3


In [38]:
grafico_base = alt.Chart(df2).encode(
    x = alt.X('Counts', title = "Comments"),
    y = alt.Y('Labels', sort = "-y")
)
grafico_base.mark_bar(opacity = 0.6)

grafico_compuesto = alt.layer(
    grafico_base.mark_text(dx=10, color = 'black').encode(text='Counts'),
    grafico_base.mark_bar(opacity = 0.6).encode(color = alt.Color('Labels', scale=alt.Scale(
            range=['red', 'yellow', 'green'])))
).properties(width = 450, height = 100, title = "Tripadvisor comments 2015-2021")
grafico_compuesto

alt.LayerChart(...)

In [40]:
df3 = df.groupby('Year').Label.count().reset_index()
df3.columns = ['Year', 'Count_Comments']
df3 = df3[:-1] #Quitamos la ultima fila porque tiene ruido una fecha
df3

,Year,Count_Comments
0,2015,7
1,2016,27
2,2017,42
3,2018,29
4,2019,34
5,2020,8
6,2021,7


In [41]:
grafico_base2 = alt.Chart(df3).encode(
    x = alt.X('Year', title = "Year"),
    y = alt.Y('Count_Comments', title = "Comments")
)
#grafico_base2.mark_text().encode(text = "Count_Comments")

grafico_compuesto2 = alt.layer(
    grafico_base2.mark_text(dy = -12, color = 'black').encode(text='Count_Comments'),
    grafico_base2.mark_line(opacity = 0.6)
).properties(width = 400, height = 250, title = "Comments per year")
grafico_compuesto2

alt.LayerChart(...)

In [42]:
df4 = df.groupby('Year').Label.value_counts().reset_index
#df4.columns = ['Year', 'Label', 'Counts']
#df4 = df4[:-1]
df4

<bound method Series.reset_index of Year  Label   
2015  positive     5
      negative     1
      neutral      1
2016  positive    26
      neutral      1
2017  positive    39
      negative     3
2018  positive    27
      negative     1
      neutral      1
2019  positive    31
      negative     3
2020  positive     8
2021  positive     5
      negative     2
trás  positive     1
Name: Label, dtype: int64>